In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.options.plotting.backend = "plotly"

import sys
sys.path.insert(0, 'driftpy/src/')

import driftpy
print(driftpy.__file__)

import numpy as np 
from sim.sim import DriftSim, SimpleDriftSim, load_hist_oracle

import os
import datetime
from sim.agents import * 
from programs.clearing_house.state import * 
from sim.events import OpenPositionEvent
from sim.helpers import random_walk_oracle, rand_heterosk_oracle, class_to_json
import pickle as cPickle

In [ ]:
def make_clearing_house(
    base_spread, 
    oracle=None,
):
    # generate random oracle data 
    if oracle is None: 
        prices, timestamps = random_walk_oracle(1)
        oracle = Oracle(prices=prices, timestamps=timestamps)
    
    # create the market with/without spread 
    amm = AMM(
        oracle=oracle, 
        base_asset_reserve=int(10_000_000 * AMM_RESERVE_PRECISION), 
        quote_asset_reserve=int(10_000_000 * AMM_RESERVE_PRECISION),
        funding_period=60*60,
        peg_multiplier=int(oracle.get_price(0) * PEG_PRECISION),
        base_spread=base_spread
    )
    market = Market(amm)
        
    # create clearing house 
    fee_structure = FeeStructure(numerator=1, denominator=1000)
    clearing_house = ClearingHouse([market], fee_structure)
    clearing_house.name = f''

    return clearing_house
    
# agent which always pushes mark => oracle 
arb = Arb(
    intensity=0.8, 
    market_index=0, 
    user_index=0,
)

# random trader 
noise = Noise(
    intensity=1, 
    market_index=0, 
    user_index=0,
    size=1_000, # 1k trades
)

ch = make_clearing_house(base_spread=0)
market = ch.markets[0]
peg = market.amm.peg_multiplier / PEG_PRECISION
sqrt_k = market.amm.sqrt_k / 1e13
full_amm_position_quote = sqrt_k * peg * 2 * 1e6

# deposit_amount = 1_000 * QUOTE_PRECISION
deposit_amount = full_amm_position_quote

lp = LP(
    lp_start_time=0, 
    lp_duration=300, 
    deposit_amount=deposit_amount, 
    user_index=1, 
    market_index=0,
)

agents = [
    arb,
    noise,
    lp,
]
os.makedirs("sim-results/", exist_ok=True)

sim = SimpleDriftSim(
    "sim-results/lp",
    clearing_house=ch,
    agents=agents
)

sim.run()
results = sim.to_df(save=True)
results.head()

In [ ]:
keep_columns = results.columns
keep_columns = [c for c in keep_columns if results[c].dtype == float or results[c].dtype == int]
# keep_columns = [c for c in keep_columns if 'u1' in c] # only lp 
filtered_df = results[keep_columns]
filtered_df.plot()

In [ ]:
results.columns

In [ ]:
users = ['arb0', 'LP1']
cols = [[user+'_'+x for x in [
    'collateral',
       # 'u0_m0_upnl', 
       'total_collateral',
       'm0_ufunding',
        'm0_upnl_noslip',
       'm0_base_asset_amount',
        'm0_quote_asset_amount',
         # 'user0.free_collateral',
       # 'user0.margin_ratio', 'user0.total_position_value'
       ]] for user in users]
flat_list = [item for sublist in cols for item in sublist]

# (results[flat_list]/1e6).iloc[5:].fillna(0).diff().cumsum().plot()
(results[flat_list]/1e6).plot()

In [ ]:
user = 'arb0'
entry_taker = ((results[user+'_m0_quote_asset_amount']*1e7)
 /abs((results[user+'_m0_base_asset_amount']))
)
user = 'LP1'
entry_maker = ((results[user+'_m0_quote_asset_amount']*1e7)
 /abs((results[user+'_m0_base_asset_amount']))
)
pd.concat([entry_maker, entry_taker],axis=1).plot()